### Wczytywanie modułów

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import keras

### Wczytywanie funkcji

In [5]:
def split_data(data, x_cols, y_col):
    """
    Split data to train/ test based on specific list of columns.
    
    Args:
        data: DataFrame.
        x_cols: List of input columns.
        y_col: Output column. 
    
    Returns:
        train_data
        test_data
        train_label
        test_label
    """
    return train_test_split(data.loc[:, x_cols], 
                            data.loc[:, y_col], test_size = 0.1,
                            random_state = 20190411)

def create_training_plot(model, metric = 'loss'):
    """
    Desc
    
    Args:
        model:
        metric:
        
    Returns:
    
    """
    plt.plot(model.history[metric])
    plt.plot(model.history['val_' + metric])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    return plt.show()

### Wczytywanie danych 

In [6]:
df = pd.read_csv('https://gitlab.com/Nicolabo/airbnbdata/raw/master/listings.csv')

### Tworzenie modelu odniesienia (baseline model)

In [7]:
"""
Create a baseline for price prediction.

It will be based on average calculated for specific neighborhood.
"""

baseline_df = df\
  .assign(neighbourhood_price = lambda x: x.groupby('neighbourhood')['price'].transform('mean'))\
  .assign(dump_diff = lambda x: np.abs(x.price - x.neighbourhood_price))

print(f'The metrics (MAE) for the baseline model is {np.mean(baseline_df.dump_diff)}.')

The metrics (MAE) for the baseline model is 69.64289719158455.


### Rozwiązanie zadań


In [8]:
# Sprawdzenie oraz wypełnienie braków danych
df.isnull().sum()

df['beds'].fillna(np.mean(df.beds),inplace=True)
df['review_scores_rating'].fillna(np.mean(df.review_scores_rating),inplace=True)

df.isnull().sum()

neighbourhood           1
latitude                0
longitude               0
number_of_reviews       0
accommodates            0
beds                    0
review_scores_rating    0
price                   0
dtype: int64

In [9]:
# Podział danych
train_x, test_x, train_y, test_y = split_data(
    df, 
    ['latitude', 'longitude', 'number_of_reviews', 'accommodates', 'beds', 'review_scores_rating'], 
    'price'
)

In [10]:
# Standaryzacja zmiennych modelu
std_scale=StandardScaler().fit(train_x)

train_x_std = std_scale.transform(train_x)
test_x_std = std_scale.transform(test_x)

In [11]:
# Definicja struktury modelu
model = keras.models.Sequential()
model.add(keras.layers.Dense(units = 256, input_dim = 6, activation='relu'))
model.add(keras.layers.Dense(units = 128, activation='relu'))
model.add(keras.layers.Dense(units = 32, activation='relu'))
model.add(keras.layers.Dense(units = 1))

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               1792      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 38,849
Trainable params: 38,849
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Definiujemy sposób uczenia
model.compile(
  loss = 'mean_squared_error',
  optimizer = 'adam',
  metrics = ['mae']
)

In [14]:
# Trening modelu
model.fit(
  x = train_x_std,
  y = train_y,
  epochs = 100,
  validation_split=0.1,
  batch_size=512
)


Train on 64071 samples, validate on 7120 samples
Epoch 1/100
64071/64071 [==============================] - 1s 18us/step - loss: 37655.5804 - mae: 79.3053 - val_loss: 72839.1955 - val_mae: 64.8215
Epoch 2/100
64071/64071 [==============================] - 1s 14us/step - loss: 31043.7799 - mae: 58.6672 - val_loss: 71985.7036 - val_mae: 61.6763
Epoch 3/100
64071/64071 [==============================] - 1s 13us/step - loss: 30582.4994 - mae: 56.7234 - val_loss: 71624.8084 - val_mae: 61.3265
Epoch 4/100
64071/64071 [==============================] - 1s 15us/step - loss: 30281.5517 - mae: 55.3749 - val_loss: 71222.2920 - val_mae: 59.6217
Epoch 5/100
64071/64071 [==============================] - 1s 16us/step - loss: 29825.4811 - mae: 53.8191 - val_loss: 70736.5606 - val_mae: 57.3727
Epoch 6/100
64071/64071 [==============================] - 1s 14us/step - loss: 29360.8048 - mae: 52.3928 - val_loss: 70276.9046 - val_mae: 61.5263
Epoch 7/100
64071/64071 [==============================] - 1s 

64071/64071 [==============================] - 1s 14us/step - loss: 25705.5114 - mae: 48.3678 - val_loss: 66738.3611 - val_mae: 52.4460
Epoch 55/100
64071/64071 [==============================] - 1s 14us/step - loss: 25556.6533 - mae: 48.2756 - val_loss: 66671.5457 - val_mae: 55.6618
Epoch 56/100
64071/64071 [==============================] - 1s 14us/step - loss: 25792.1697 - mae: 48.9968 - val_loss: 66843.6737 - val_mae: 51.8221
Epoch 57/100
64071/64071 [==============================] - 1s 14us/step - loss: 25511.6705 - mae: 48.3100 - val_loss: 66633.8570 - val_mae: 53.2939
Epoch 58/100
64071/64071 [==============================] - 1s 15us/step - loss: 25526.3342 - mae: 48.4673 - val_loss: 66332.0855 - val_mae: 53.4128
Epoch 59/100
64071/64071 [==============================] - 1s 14us/step - loss: 25564.3825 - mae: 48.1865 - val_loss: 66502.8058 - val_mae: 55.4613
Epoch 60/100
64071/64071 [==============================] - 1s 14us/step - loss: 25439.3378 - mae: 48.5010 - val_loss: 

In [15]:
model.evaluate(test_x_std, test_y)

7911/7911 [==============================] - 0s 25us/step


[71702.03311125208, 49.86172866821289]